<a href="https://colab.research.google.com/github/overfit-ir/persian-twitter-ner/blob/feature%2Fadd-multi-task-learning/multi_task_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Get Data

In [1]:
!rm run_tf_ner.py tasks.py utils_ner.py

rm: cannot remove 'run_tf_ner.py': No such file or directory
rm: cannot remove 'tasks.py': No such file or directory
rm: cannot remove 'utils_ner.py': No such file or directory


In [2]:
!wget -q --show-progress --no-cache https://raw.githubusercontent.com/overfit-ir/persian-twitter-ner/feature/add-multi-task-learning/run_tf_ner.py
!wget -q --show-progress --no-cache https://raw.githubusercontent.com/overfit-ir/persian-twitter-ner/feature/add-multi-task-learning/tasks.py
!wget -q --show-progress --no-cache https://raw.githubusercontent.com/overfit-ir/persian-twitter-ner/feature/add-multi-task-learning/utils_ner.py

run_tf_ner.py       100%[===================>]  15.16K  --.-KB/s    in 0s      
tasks.py            100%[===================>]   5.38K  --.-KB/s    in 0s      
utils_ner.py        100%[===================>]  34.07K  --.-KB/s    in 0s      


In [3]:
!wget -q --show-progress https://raw.githubusercontent.com/overfit-ir/persian-twitter-ner/master/twitter_data/persian-ner-twitter-data/train.txt
!wget -q --show-progress https://raw.githubusercontent.com/overfit-ir/persian-twitter-ner/master/twitter_data/persian-ner-twitter-data/test.txt
!wget -q --show-progress https://raw.githubusercontent.com/overfit-ir/persian-twitter-ner/master/twitter_data/persian-ner-twitter-data/dev.txt
!mkdir data_twitter && mv train.txt data_twitter && mv test.txt data_twitter && mv dev.txt data_twitter

train.txt           100%[===================>]   2.20M  --.-KB/s    in 0.03s   
test.txt            100%[===================>] 108.10K  --.-KB/s    in 0.002s  
dev.txt             100%[===================>] 160.83K  --.-KB/s    in 0.005s  


In [4]:
!wget -q --show-progress https://raw.githubusercontent.com/overfit-ir/persian-twitter-ner/feature/add-multi-task-learning/ner_data/peyma/dev.txt
!wget -q --show-progress https://raw.githubusercontent.com/overfit-ir/persian-twitter-ner/feature/add-multi-task-learning/ner_data/peyma/train.txt
!wget -q --show-progress https://raw.githubusercontent.com/overfit-ir/persian-twitter-ner/feature/add-multi-task-learning/ner_data/peyma/test.txt
!mkdir data_peyma && mv train.txt data_peyma && mv test.txt data_peyma && mv dev.txt data_peyma

dev.txt             100%[===================>] 297.21K  --.-KB/s    in 0.008s  
train.txt           100%[===================>]   2.64M  --.-KB/s    in 0.03s   
test.txt            100%[===================>] 359.39K  --.-KB/s    in 0.007s  


### Install Dependencies

In [5]:
!pip -q install transformers
!pip -q install sentencepiece
!pip -q install seqeval
!pip -q install conllu

     |████████████████████████████████| 2.0MB 24.0MB/s 
     |████████████████████████████████| 3.2MB 55.3MB/s 
     |████████████████████████████████| 890kB 51.9MB/s 
     |████████████████████████████████| 1.2MB 20.8MB/s 
     |████████████████████████████████| 51kB 7.5MB/s 


In [6]:
import transformers
transformers.__version__

'4.4.2'

### Preprocess

In [7]:
!wget -q --show-progress "https://raw.githubusercontent.com/stefan-it/fine-tuned-berts-seq/master/scripts/preprocess.py"

preprocess.py       100%[===================>]     991  --.-KB/s    in 0s      


#### Preprocess twitter data

In [8]:
!cat data_twitter/train.txt | grep -v "^#" | cut -f 1,2 | tr '\t' ' ' > data_twitter/train.txt.tmp
!cat data_twitter/test.txt | grep -v "^#" | cut -f 1,2 | tr '\t' ' ' > data_twitter/test.txt.tmp
!cat data_twitter/dev.txt | grep -v "^#" | cut -f 1,2 | tr '\t' ' ' > data_twitter/dev.txt.tmp

In [9]:
!mkdir processed_data_twitter

In [10]:
!python3 preprocess.py data_twitter/dev.txt.tmp HooshvareLab/bert-base-parsbert-uncased 256 > processed_data_twitter/dev.txt
!python3 preprocess.py data_twitter/train.txt.tmp HooshvareLab/bert-base-parsbert-uncased 256 > processed_data_twitter/train.txt
!python3 preprocess.py data_twitter/test.txt.tmp HooshvareLab/bert-base-parsbert-uncased 256 > processed_data_twitter/test.txt

Downloading: 100% 434/434 [00:00<00:00, 653kB/s]
Downloading: 100% 1.22M/1.22M [00:00<00:00, 36.9MB/s]


#### Preprocess Peyma

In [11]:
with open('data_peyma/train.txt') as file:
  content = ''
  for line in file.readlines():
    content += line.replace(' ', '\u200c')
with open('data_peyma/train.txt.tmp', 'w') as file:
  file.write(content)

In [12]:
with open('data_peyma/test.txt') as file:
  content = ''
  for line in file.readlines():
    content += line.replace(' ', '\u200c')
with open('data_peyma/test.txt.tmp', 'w') as file:
  file.write(content)

In [13]:
with open('data_peyma/dev.txt') as file:
  content = ''
  for line in file.readlines():
    content += line.replace(' ', '\u200c')
with open('data_peyma/dev.txt.tmp', 'w') as file:
  file.write(content)

In [14]:
!cat data_peyma/train.txt.tmp | grep -v "^#" | cut -f 1,2 | tr '|' ' ' > data_peyma/train.txt.tmp2
!cat data_peyma/test.txt.tmp | grep -v "^#" | cut -f 1,2 | tr '|' ' ' > data_peyma/test.txt.tmp2
!cat data_peyma/dev.txt.tmp | grep -v "^#" | cut -f 1,2 | tr '|' ' ' > data_peyma/dev.txt.tmp2

In [15]:
!mkdir processed_data_peyma

In [16]:
!python3 preprocess.py data_peyma/dev.txt.tmp2 HooshvareLab/bert-base-parsbert-uncased 256 > processed_data_peyma/dev.txt
!python3 preprocess.py data_peyma/train.txt.tmp2 HooshvareLab/bert-base-parsbert-uncased 256 > processed_data_peyma/train.txt
!python3 preprocess.py data_peyma/test.txt.tmp2 HooshvareLab/bert-base-parsbert-uncased 256 > processed_data_peyma/test.txt

### Retrieve Labels

In [17]:
!cat processed_data_twitter/train.txt processed_data_twitter/test.txt processed_data_twitter/dev.txt | cut -d " " -f 2 | grep -v "^$"| sort | uniq > labels_twitter.txt

In [18]:
!cat processed_data_peyma/train.txt processed_data_peyma/test.txt processed_data_peyma/dev.txt | cut -d " " -f 2 | grep -v "^$"| sort | uniq > labels_peyma.txt

### Multi-task Training

In [ ]:
!python3 run_tf_ner.py \
--data_dir_twitter ./processed_data_twitter/ \
--data_dir_peyma ./processed_data_peyma/ \
--labels_twitter labels_twitter.txt \
--labels_peyma labels_peyma.txt \
--model_name_or_path HooshvareLab/bert-base-parsbert-uncased \
--output_dir eval/ \
--max_seq_length_twitter  256 \
--max_seq_length_peyma  256 \
--per_device_train_batch_size 8 \
--save_steps 1000 \
--save_total_limit 1 \
--eval_steps 100 \
--logging_steps 100 \
--logging_first_step True \
--evaluation_strategy steps \
--seed 1 \
--num_train_epochs 5 \
--do_train \
--do_eval \
--do_predict

2021-03-24 09:40:35.840225: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
[INFO|training_args.py:631] 2021-03-24 09:40:37,022 >> PyTorch: setting up devices
[INFO|training_args.py:555] 2021-03-24 09:40:37,090 >> The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
[INFO|training_args_tf.py:192] 2021-03-24 09:40:37,095 >> Tensorflow: setting up strategy
2021-03-24 09:40:37.099986: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-03-24 09:40:37.100325: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-03-24 09:40:37.100633: I tensorflow/stream_executor/cuda/cuda_gpu_executor.c